# GPU-Jupyter

This Jupyterlab Instance is connected to the GPU via CUDA drivers. In this notebook, we test the installation and perform some basic operations on the GPU.

## Test GPU connection

#### Using the following command, your GPU type and its NVIDIA-SMI driver version should be listed:

In [1]:
!nvidia-smi

Tue Dec 19 15:20:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.54       Driver Version: 526.56       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P0    19W /  N/A |      0MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Now, test if PyTorch can access the GPU via CUDA:

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import tensorflow as tf
print(tf.test.is_built_with_cuda(), tf.config.list_physical_devices('GPU'))

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2023-12-19 15:20:31.292167: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 15:20:31.292283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 15:20:31.345027: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-19 15:20:31.405681: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[]


2023-12-19 15:20:33.437438: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-19 15:20:33.438375: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-19 15:20:33.438401: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-19 15:20:33.439883: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-19 15:20:33.439914: I external/local_xla/xla/stream_executor

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16132906446837489126
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3153068032
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13564847057713934038
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [4]:
from __future__ import print_function
import numpy as np
import torch
a = torch.rand(5, 3)
a

tensor([[0.6527, 0.7924, 0.8878],
        [0.7822, 0.0338, 0.4025],
        [0.6130, 0.1177, 0.9255],
        [0.0451, 0.7010, 0.0331],
        [0.1705, 0.8165, 0.7323]])

## Performance test

#### Now we want to know how much faster a typical operation is using GPU. Therefore we do the same operation in numpy, PyTorch and PyTorch with CUDA. The test operation is the calculation of the prediction matrix that is done in a linear regression.

### 1) Numpy

In [5]:
x = np.random.rand(10000, 256)

In [6]:
%%timeit
H = x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose())

559 ms ± 87.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2) PyTorch

In [7]:
x = torch.rand(10000, 256)

In [8]:
%%timeit
# Calculate the projection matrix of x on the CPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

200 ms ± 4.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 3) PyTorch on GPU via CUDA

In [9]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 256, device=device) # directly create a tensor on GPU
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])
    print(y.to("cpu", torch.double)[0:5, 0:5])

tensor([[0.4609, 0.7584, 0.4593, 0.0551, 0.1594],
        [0.6063, 0.5960, 0.4197, 0.7962, 0.1542],
        [0.5160, 0.4067, 0.6062, 0.1356, 0.8867],
        [0.3636, 0.7090, 0.3487, 0.0552, 0.4904],
        [0.6309, 0.0065, 0.8926, 0.0643, 0.6346]], device='cuda:0')
tensor([[0.4609, 0.7584, 0.4593, 0.0551, 0.1594],
        [0.6063, 0.5960, 0.4197, 0.7962, 0.1542],
        [0.5160, 0.4067, 0.6062, 0.1356, 0.8867],
        [0.3636, 0.7090, 0.3487, 0.0552, 0.4904],
        [0.6309, 0.0065, 0.8926, 0.0643, 0.6346]], dtype=torch.float64)


In [10]:
%%timeit
# Calculate the projection matrix of x on the GPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

The slowest run took 5.26 times longer than the fastest. This could mean that an intermediate result is being cached.
90.5 ms ± 74.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Exhaustive Testing on GPU

In [11]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 10, device=device) # directly create a tensor on GPU

In [12]:
if torch.cuda.is_available():
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])

tensor([[0.2782, 0.4737, 0.6745, 0.9081, 0.3480],
        [0.1753, 0.6475, 0.3926, 0.3947, 0.9197],
        [0.1747, 0.6550, 0.4903, 0.4221, 0.3066],
        [0.8606, 0.6053, 0.0784, 0.1127, 0.2536],
        [0.0236, 0.5264, 0.6400, 0.5198, 0.5281]], device='cuda:0')


In [13]:
if torch.cuda.is_available():
    # Here is the memory of the GPU a border.
    # A matrix with 100000 lines requires 37 GB, but only 8 GB are available.
    H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

In [14]:
if torch.cuda.is_available():
    print(H[0:5, 0:5])

tensor([[ 6.5489e-04,  1.8794e-04,  2.2678e-04, -2.8653e-04,  1.9096e-04],
        [ 1.8794e-04,  7.0443e-04,  2.0275e-04, -2.2673e-04,  2.6704e-04],
        [ 2.2678e-04,  2.0275e-04,  6.8227e-04, -1.5024e-05,  3.2128e-04],
        [-2.8653e-04, -2.2673e-04, -1.5024e-05,  1.1865e-03,  1.9364e-04],
        [ 1.9096e-04,  2.6704e-04,  3.2128e-04,  1.9364e-04,  1.0109e-03]],
       device='cuda:0')


In [15]:
if torch.cuda.is_available():
    # This operation is difficult, as an symmetric matrix is transferred
    # back to the CPU. Is possible up to 30000 rows.
    print(H.to("cpu", torch.double)[0:5, 0:5])

tensor([[ 6.5489e-04,  1.8794e-04,  2.2678e-04, -2.8653e-04,  1.9096e-04],
        [ 1.8794e-04,  7.0443e-04,  2.0275e-04, -2.2673e-04,  2.6704e-04],
        [ 2.2678e-04,  2.0275e-04,  6.8227e-04, -1.5024e-05,  3.2128e-04],
        [-2.8653e-04, -2.2673e-04, -1.5024e-05,  1.1865e-03,  1.9364e-04],
        [ 1.9096e-04,  2.6704e-04,  3.2128e-04,  1.9364e-04,  1.0109e-03]],
       dtype=torch.float64)
